# 第2回講義 宿題

## 課題
今Lessonで学んだことを元に、MNISTのファッション版 (Fashion MNIST、クラス数10) をソフトマックス回帰によって分類してみましょう。

Fashion MNISTの詳細については以下のリンクを参考にしてください。

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`lecture02`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`lecture02 (code)`を選択して**提出してください。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します。
- 定時に採点しLeader Boardを更新します。(採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします。

### データの読み込み (このセルは修正しないでください)

In [2]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('drive/My Drive/Colab Notebooks/DLBasics2022_colab/Lecture02/data/x_train.npy')
    y_train = np.load('drive/My Drive/Colab Notebooks/DLBasics2022_colab/Lecture02/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('drive/My Drive/Colab Notebooks/DLBasics2022_colab/Lecture02/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

In [3]:
x_train, y_train, x_test = load_fashionmnist()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ソフトマックス回帰の実装

In [4]:
x_train, y_train, x_test = load_fashionmnist()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def softmax(x):
    x -= x.max(axis=1, keepdims=True) # expのoverflowを防ぐ
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)
  
def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=1e+10))

def cross_entropy(y,y_hat):
    return (- y * np_log(y_hat)).sum(axis=1).mean()
# weights
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')
b = np.zeros(shape=(10,)).astype('float32')

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, y, eps=1.0):
    # WRITE ME
    global W, b
    
    batch_size = x.shape[0]
    
    # 予測
    y_hat = softmax(np.matmul(x, W) + b) # shape: (batch_size, 出力の次元数)
    
    # 目的関数の評価
    cost = cross_entropy(y,y_hat)
    delta = y_hat - y # shape: (batch_size, 出力の次元数)
    
    # パラメータの更新
    dW = np.matmul(x.T, delta) / batch_size # shape: (入力の次元数, 出力の次元数)
    db = np.matmul(np.ones(shape=(batch_size,)), delta) / batch_size # shape: (出力の次元数,)
    W -= eps * dW
    b -= eps * db
    return cost

def valid(x, y):
    # WRITE ME
    y_hat=softmax(np.matmul(x, W) + b)
    cost=cross_entropy(y,y_hat)
    return cost,y_hat



In [12]:
eps=1.0
for epoch in range(1000):
    # オンライン学習
    # WRITE ME
    x_train, y_train = shuffle(x_train, y_train)
    cost = train(x_train, y_train,min(1,10/(10+3000+(epoch//100)*100)**(1.0/2.0)))
    cost, y_pred = valid(x_valid, y_valid)
    if epoch % 10 == 9 or epoch == 0:
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
            epoch + 1,
            cost,
            accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
        ))

y_pred =softmax(np.matmul(x_test, W) + b).argmax(axis=1)# WRITE ME


submission = pd.Series(y_pred, name='label')
submission.to_csv('drive/My Drive/Colab Notebooks/DLBasics2022_colab/Lecture02/submission_pred.csv', header=True, index_label='id')

EPOCH: 1, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 10, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 20, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 30, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 40, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 50, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 60, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 70, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 80, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 90, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 100, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 110, Valid Cost: 0.414, Valid Accuracy: 0.864
EPOCH: 120, Valid Cost: 0.414, Valid Accuracy: 0.863
EPOCH: 130, Valid Cost: 0.414, Valid Accuracy: 0.863
EPOCH: 140, Valid Cost: 0.414, Valid Accuracy: 0.863
EPOCH: 150, Valid Cost: 0.414, Valid Accuracy: 0.863
EPOCH: 160, Valid Cost: 0.414, Valid Accuracy: 0.863
EPOCH: 170, Valid Cost: 0.414, Valid Accuracy: 0.863
EPOCH: 180, Valid Cost: 0.414, Valid Accuracy: 0.863
EPOC

In [ ]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(cross_entropy(np.array([[1,0],[0,1]]),np.array([[1,0],[0,1]])))
print(x.shape[0])

(54000, 784)
(54000, 10)
(10000, 784)
-0.0
